<a href="https://colab.research.google.com/github/basharE/CVDs_Predictors/blob/main/CVDs_Predictors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages/Imports

In [2]:
!pip install -U -q PyDrive
!pip install shap
!pip install boruta
!pip install BorutaShap
!pip install catboost
!pip install verstack

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2
from scipy.special import softmax
from boruta import BorutaPy
from BorutaShap import BorutaShap
from verstack import LGBMTuner
from sklearn.model_selection import cross_val_score

import pandas as pd
import numpy as np
import seaborn as sn
sn.set_style(style='darkgrid')
import matplotlib.pyplot as plt
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 7.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 3.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 7.5 MB/s 
     |████████████████████████████████| 23.2 MB 86.8 MB/s 
     |████████████████████████████████| 2.0 MB 51.0 MB/s 
     |████████████████████████████████| 308 kB 74.1 MB/s 
     |████████████████████████████████| 23.9 MB 1.3 MB/s 
     |████████████████████████████████| 227 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Uploading data 

## Connecting to google drive

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Uploading csv files

In [13]:
df_Cleveland = pd.read_csv("/content/drive/MyDrive/CVD_DATA/cleveland.data", delimiter=" ")
df_Hungarian = pd.read_csv("/content/drive/MyDrive/CVD_DATA/hungarian.data", delimiter=" ")
df_Switzerland = pd.read_csv("/content/drive/MyDrive/CVD_DATA/switzerland.data", delimiter=" ")
df_LongBeachVA = pd.read_csv("/content/drive/MyDrive/CVD_DATA/long-beach-va.data", delimiter=" ")

In [18]:
df_LongBeachVA

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,...,rcaprox,rcadist,lvx1,lvx2,vx3,lvx4,lvf,cathef,junk,name
0,1,0,63,1,1,1,1,-9,4,140,...,2,1,1,1,1,1,1,0.70,5.5,name
1,2,0,44,1,1,1,1,-9,4,130,...,1,1,1,1,1,1,1,0.50,-9.0,name
2,3,0,60,1,1,1,1,-9,4,132,...,2,1,1,1,1,7,2,0.52,4.1,name
3,4,0,55,1,1,1,1,-9,4,142,...,1,1,1,1,1,1,1,0.73,6.5,name
4,5,0,66,1,1,0,0,-9,3,110,...,1,1,1,1,1,1,1,0.73,8.0,name
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,200,0,54,0,1,1,1,-9,4,127,...,1,1,1,1,1,1,1,0.76,5.6,name
196,201,0,62,1,0,0,0,-9,1,-9,...,1,1,1,1,1,1,2,0.62,3.5,name
197,202,0,55,1,1,1,1,-9,4,122,...,2,1,1,1,1,1,1,0.69,5.6,name
198,116,0,58,1,1,1,1,-9,4,-9,...,1,1,1,1,1,1,1,0.81,6.0,name
